In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
py.init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore')
import time,datetime
import gc

total_Exposure_Log_path = ''
# user_data_path = 'https://lwh-tenxun-test.oss-cn-shenzhen.aliyuncs.com/testA/user_data.csv'

Btest_sample_new_path = ''

In [0]:
Btest_sample = pd.read_csv(Btest_sample_new_path,sep='\t',names=['Sample_id','ad_id','Creation_time','ad_size','ad_industry_id','Product_Types',
                                                           'Product_id','ad_account_id','Delivery_period','Crowd_orientation','ad_bid'])
# Btest_sample['Creation_time'] = pd.to_datetime(Btest_sample['Creation_time'],unit='s')
Btestlist = Btest_sample.ad_id.unique()

In [0]:
total_Exposure_Log = pd.read_csv(total_Exposure_Log_path,sep='\t',usecols = [1,4,6],
                                 names = ['ad_request_time','ad_id','bid'],header = None,iterator = True,low_memory = False)
n = 3000
temp = pd.DataFrame()
for i in range(n):
    try:
        log = total_Exposure_Log.get_chunk(1000000)
        log.drop_duplicates(keep = 'first',inplace = True)
        log = log[log['ad_id'].isin(Btestlist)]
        if (temp.empty):
            temp = log
        else:
            temp = temp.append(log,ignore_index = True)
    except StopIteration:
        break
temp['ad_request_time'] = pd.to_datetime(temp['ad_request_time'],unit='s')

In [0]:
temp['month'] = temp['ad_request_time'].dt.month
temp['day'] = temp['ad_request_time'].dt.day

In [0]:
y_train = temp.groupby(['ad_id','month','day']).size().reset_index(name='target')
# y_train = y_train.drop(['month','day'],axis=1)

In [0]:
temp = temp.drop(['month','day'],axis = 1)
ad_bid = temp['bid'].groupby([temp['ad_id']]).mean().reset_index(name='ad_bid')
temp = temp.drop(['bid'],axis = 1)
ad_bid.head()

In [0]:
train1 = y_train.merge(ad_bid, on='ad_id', how='left')

In [0]:
train1.to_csv('targrtB_data.csv', index=False)